# Multi-dimensional datasets

This is the continuation of [1-D datasets and tables](introduction.ipynb).

## Creation, slicing, and visualization

In [ ]:
import numpy as np
import scipp as sc
from scipp import Dim

To create variables with more than one dimension we specify a list of dimension labels and provide data with a corresponding shape.
When inserted into a dataset it is important to note that while the dimensions extents have to match, individual variables may have transposed memory layout.

In [ ]:
d = sc.Dataset(
    {
    'alice': sc.Variable([Dim.Z, Dim.Y, Dim.X], values=np.random.rand(10, 10, 10), variances=0.1*np.random.rand(10, 10, 10)),
    'bob': sc.Variable([Dim.X, Dim.Z], values=np.arange(0.0, 10.0, 0.1).reshape(10, 10), variances=0.1*np.random.rand(10, 10))
    },
    coords={
        Dim.X: sc.Variable([sc.Dim.X], values=np.arange(11.0), unit=sc.units.m),
        Dim.Y: sc.Variable([sc.Dim.Y], values=np.arange(11.0), unit=sc.units.m),
        Dim.Z: sc.Variable([sc.Dim.Z], values=np.arange(11.0), unit=sc.units.m)})

Note that in this example the coordinates are exceeding the shape of the data by 1.
 This means that the coordinates represent bin edges:

In [ ]:
sc.show(d)
print(d)

To slice in multiple dimensions, we can simply chain the slicing notation used previously for 1D data.
This gives us a number of different options for visualizing our data:

In [ ]:
sc.table(d[Dim.X, 5][Dim.Z, 2])

We can plot and item of a `Dataset` using:

In [ ]:
# Remove this line to get plotly plots (currently not available on readthedocs)
sc.plot_config.backend = "matplotlib"
sc.plot(d["bob"])

We can also plot the standard deviations, i.e., the square root of the variances stored in the dataset, alongside the values with:

In [ ]:
sc.plot(d["bob"], show_variances=True)

Plotting a 3-dimensional data cube will show a 2D image with a slider to navigate through the third dimension:

In [ ]:
sc.plot(d["alice"], backend='plotly') # This currently fails on readthedocs due to https://github.com/readthedocs/sphinx_rtd_theme/issues/788

Finally, by extracting a 1D variable, we obtain a 1D plot:

In [ ]:
sc.plot(d[Dim.X, 8][Dim.Y, 2])

Note that this is now plotted as a histogram since the coordinate in the dataset represents bin edges, in contrast to the 1D data plotted in [1-D datasets and tables](introduction.ipynb).

Operations automatically broadcast based on dimension labels.
That is, if one of the operands lacks one (or multiple) dimensions that the other operands have, the operand is considered constant along those dimensions.
Its values are implicitly "duplicated" so the shape matches.
In contrast to `numpy` or `MATLAB` there is no need to keep track of dimension order:

In [ ]:
d["alice"] -= d["bob"]
d["alice"] -= d["alice"][Dim.Y, 5]
sc.plot(d["alice"][Dim.X, 4])

### Exercise 1

Remove the X and Z surface layer of the volume, i.e., remove the first and last slice in each of the dimensions `Dim.X` and `Dim.Z`.

### Solution 1

In [ ]:
d = d[Dim.X, 1:-1][Dim.Z, 1:-1].copy()
d

Note the important call to `copy()`.
If we omit it, `d` will just be a multi-dimensional slice of the larger volume (which is kept alive), wasting memory and preventing further modification, such as insertion of other variables.

Note also that if we had also sliced `Dim.Y` the result would not contain the data for `'bob'` since this item does not depend on Y.

## More advanced operations with multi-dimensional datasets
Operations like `concatenate` and `merge` work just like with one-dimensional datasets.

### Exercise 2
- Try to concatenate the dataset with itself along the X dimensions. Why does this fail?
- Make a copy of the dataset, add an offset to the X coordinate to fix the issue, and try to concatenate again.

### Solution 2

In [ ]:
try:
    d = sc.concatenate(d, d, Dim.X)
except RuntimeError:
    print("Failed as expected!")

With a data extent of, e.g. `8` in this case, bin edges have extent `9`.
Naive concatenation would thus lead a new data extent of `16` and a coordinate extent of `18`, which is meaningless and thus prevented.
In this `concatenate` merges the last edge of the first input with the first edge of the second input, if compatible.

In [ ]:
offset = d.copy()
offset.coords[Dim.X] += sc.Variable(8.0, unit=sc.units.m)
combined = sc.concatenate(d, offset, Dim.X)
sc.plot(combined['alice'], backend='plotly')

Another available operation is `rebin`.
 This is only for count-data or count-density-data, so we have to set an appropriate unit first:

In [ ]:
d['alice'].unit = sc.units.counts
d['bob'].unit = sc.units.counts

Before `rebin` we have the following:

In [ ]:
print(d)
sc.plot(d[Dim.Z, 0])

We rebin onto a coarser grid, in this case combining two neightboring bins:

In [ ]:
new_x = sc.Variable([Dim.X], values=d.coords[Dim.X].values[::2])
d = sc.rebin(d, Dim.X, new_x)

The result looks as follows:

In [ ]:
print(d)
sc.plot(d[Dim.Z, 0])

## Interaction with `numpy`

Variable in a dataset are exposed in a `numpy`-compatible buffer format, so we can directly hand them to `numpy` functions:

In [ ]:
d['alice'].values = np.sin(d['alice'].values)

In contrast to the 1-D case considered earlier, the `values` are now a multi-dimensional array:

In [ ]:
d['alice'].values

### Exercise 3
 1. Use `sc.mean` to compute the mean of the data for Alice along the Z dimension.
 2. Do the same with `numpy`, what are the complications you encounter, that are not present when using the dataset?

### Solution 3

In [ ]:
help(sc.mean)

In [ ]:
mean = sc.mean(d['alice'], Dim.Z)

When using `numpy` to compute the mean:
- We must remember (or lookup) which dimension corresponds to the Z dimensions.
- We need a separate call for values and variances.
- We need to manually scale the variance with the inverse of the number of data points to get the variance of the mean (standard deviation of the mean scales with `1/sqrt(N)`).

In [ ]:
np_values = np.mean(d['alice'].values, axis=0)
np_variances = np.mean(d['alice'].variances, axis=0)
np_variances /= d['alice'].shape[0]

In [ ]:
print(mean.values)
print(mean.variances)
print(np_values)
print(np_variances)

Continue to [Part 3 - Neutron data](neutron-data.ipynb) to see how datasets are used with neutron-event data.